In [5]:
def clearall():
    all = [var for var in globals() if var[0] != "_"]
    for var in all:
        del globals()[var]
clearall()

In [6]:
import warnings
warnings.filterwarnings("ignore")

In [39]:
print('Importing Librairies...')
import warnings
warnings.filterwarnings("ignore")
import talib
import numpy as np
import pandas as pd
import pandas_datareader as web
from colorama import Fore, Back, Style
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from sklearn.preprocessing import MinMaxScaler
import yaml
from keras.models import model_from_yaml
from sklearn.metrics import  precision_score,accuracy_score,recall_score

import sys

import xgboost as xgb
from xgboost import XGBRegressor, plot_importance
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn')
import time
import datetime as dt
import os
print('Librairies imported')
print('')

___Author___='LumberJack Jyss'
print('Global Optimized LumberJack Environment Motor 55\nLumberJack Jyss 5780(c)')
print(Fore.BLUE,'°0Oo_D.A.G._26_oO0°')
print(Fore.YELLOW,Back.BLUE,'UNIQUE BT v0.55',Style.RESET_ALL)

print('')

LaDate = input('Entrez la date YYYY-MM-JJ de fin backtest')
LeNom = input('Quel nom?')
adjusted = input('On travaille sur adjusted (oui) ou sur close (non)')
start = LaDate[:2]+str(int(LaDate[2:4]) - 4)+'-'+LaDate[5:]
stamp = LaDate[:2]+str(int(LaDate[2:4]) + 1)+'-'+LaDate[5:]
end = LaDate[:2]+str(int(LaDate[2:4]) + 1)+'-'+LaDate[5:]

    
if adjusted == 'oui' or adjusted == 'Oui' or adjusted == 'O' or adjusted == 'o' or adjusted == 'Y' or adjusted == 'y' or adjusted == 'Yes' or adjusted == 'yes':
    print(Fore.GREEN,'On travaille sur Adjusted',Style.RESET_ALL)
    adjusted = 1
    LeNom = LeNom+'_ADJUSTED/'
    LeChemin = 'ADJUSTED/DL_'+stamp+'/'
elif adjusted == 'non' or adjusted == 'Non' or adjusted == 'N' or adjusted == 'n' or adjusted == 'no' or adjusted == 'No':
    print(Fore.GREEN,'On travaille sur Close',Style.RESET_ALL)
    adjusted = 0
    LeNom = LeNom+'_CLOSE/'
    LeChemin = 'CLOSE/DL_'+stamp+'/'
else:
    print(Fore.GREEN,'Ca dépend, ça dépasse. Les calculs sont pas bon Kevin. On passe en ajusted par défaut',Style.RESET_ALL)
    adjusted = 1
    LeNom = LeNom+'_ADJUSTED/'
    LeChemin = 'ADJUSTED/DL_'+stamp+'/'

    
try:
    os.mkdir('RESULTS')
except:
    pass
try:
    os.mkdir('RESULTS/'+LeNom)
except:
    pass

time.sleep(5)
    
def kill():
    os._exit
    return()

def generbaz():
    print(Fore.YELLOW,'ON GENERE LaBase',Style.RESET_ALL)
    tmps55=time.time()
    LaBase = pd.DataFrame()
    df = pd.DataFrame()
    BADTICKER = []
    long = len(constituents)
    for loop in range(0, long):
    #for loop in range(0, 2):
        if loop == 1:
            print('\r',Fore.GREEN,' |==',Fore.BLUE,'O',Fore.RED,'==================|',end='',flush=True)
        if loop == round(long/9):
            print('\r',Fore.GREEN,' |====',Fore.BLUE,'O',Fore.RED,'================|',end='',flush=True)
        if loop == round(long/9)*2:
            print('\r',Fore.GREEN,' |======',Fore.BLUE,'O',Fore.RED,'==============|',end='',flush=True)
        if loop == round(long/9)*3:
            print('\r',Fore.GREEN,' |========',Fore.BLUE,'O',Fore.RED,'============|',end='',flush=True)
        if loop == round(long/9)*4:
            print('\r',Fore.GREEN,' |==========',Fore.BLUE,'O',Fore.RED,'==========|',end='',flush=True)
        if loop == round(long/9)*5:
            print('\r',Fore.GREEN,' |============',Fore.BLUE,'O',Fore.RED,'========|',end='',flush=True)
        if loop == round(long/9)*6:
            print('\r',Fore.GREEN,' |==============',Fore.BLUE,'O',Fore.RED,'======|',end='',flush=True)
        if loop == round(long/9)*7:
            print('\r',Fore.GREEN,' |================',Fore.BLUE,'O',Fore.RED,'====|',end='',flush=True)
        if loop == round(long/9)*8:
            print('\r',Fore.GREEN,' |==================',Fore.BLUE,'O',Fore.RED,'==|',end='',flush=True)

        ticker = (constituents.iloc[loop]['Symb.'])
        try:
            df = web.DataReader(ticker,'yahoo',start,end)
            df['Date'] = str(df.index[0])[:10]
        except:
            BADTICKER.append(ticker)
            continue
        
        if adjusted == 1:
            
            df = df.drop(['Close'],axis=1)
            df['Close'] = df['Adj Close']
            df = df.drop(['Adj Close'],axis = 1)
            df = df.iloc[-820:,:]
            
            
        elif adjusted == 0:
            df = df.drop(['Adj Close'], axis=1)
            df = df[['High', 'Low', 'Open', 'Volume', 'Close']]
            df = df.iloc[-820:,:]
            
        
        df['Symbol'] = ticker
        df = df[['Symbol','High','Low','Open','Volume','Close']]
            
            
        LaBase = pd.concat((LaBase,df),ignore_index=False)
    tmps22=round(time.time()-tmps55,2)
    print(Fore.BLUE,"\n Time for completing LaBase Generation = %f" %tmps22,'seconds \n',Style.RESET_ALL)

    print('Il y a ',len(BADTICKER),' mauvais tickers \n')
    print('Les voici : ')
    print(BADTICKER)


    print('Base générée : ',LaBase.shape)
    print(LaBase.tail(1))
    print('Il y a ',LaBase[LaBase.duplicated()].shape,' Duplicats')
    LaBase = LaBase.drop_duplicates()
    print('\n \n Base déclonée : ',LaBase.shape)
    print(LaBase.tail(1))
    try:
        os.remove('RESULTS/'+LeNom+'LaBase_'+stamp+'.csv')
    except:
        pass
    LaBase.to_csv('RESULTS/'+LeNom+'LaBase_'+stamp+'.csv')
    return(LaBase)

def prepa_data(df):
    rsi = talib.RSI(df['Close'],timeperiod=14)
    stoc_slowk, stoc_slowd = talib.STOCH(df['High'],df['Low'],df['Close'])
    upper, middle, lower =  talib.BBANDS(df['Close'], timeperiod=9, nbdevup=2, nbdevdn=2,matype=0)
    sma5 = talib.SMA(df['Close'],timeperiod=5)
    sma8 = talib.SMA(df['Close'],timeperiod=8)
    sma10 = talib.SMA(df['Close'],timeperiod=10)
    sma12 = talib.SMA(df['Close'],timeperiod=12)
    sma15 = talib.SMA(df['Close'],timeperiod=15)
    sma30 = talib.SMA(df['Close'],timeperiod=30)
    sma35 = talib.SMA(df['Close'],timeperiod=35)
    sma40 = talib.SMA(df['Close'],timeperiod=40)
    sma45 = talib.SMA(df['Close'],timeperiod=45)
    sma50 = talib.SMA(df['Close'],timeperiod=50)
    atr = talib.ATR(df['High'],df['Low'],df['Close'],timeperiod=10)
    delta5_8 = sma5 - sma8
    delta8_10 = sma8 - sma10
    delta10_12 = sma10 - sma12
    delta12_15 = sma12 - sma15
    delta15_30 = sma15 - sma30
    delta30_35 = sma30 - sma35
    delta35_40 = sma35 - sma40
    delta40_45 = sma40 - sma45
    delta45_50 = sma45 - sma50
    bbdelta = upper - middle
    price_bolup = df['Close'] - lower
    price_bolow = df['Close'] - upper
    Ema = talib.EMA(df['Close'],timeperiod=20)
    KC_High = Ema + 2*atr
    KC_Low = Ema - 2*atr
    aroondown, aroonup = talib.AROON(df['High'], df['Low'], timeperiod=9)
    aroon = aroonup - aroondown 
    rsi30_list = []
    rsi70_list = []
    for i in range(0,df.shape[0]):
        rsi70_list.append(70 - rsi[i])
        rsi30_list.append(rsi[i] - 30)
        
    varop_spy = df['Open'] - df['Close']
    varhl_spy = df['High'] - df['Low']
    df['Varop_Spy'] = varop_spy
    df['Varhl_spy'] = varhl_spy
    df['RSI'] = rsi
    df['70 - RSI'] = np.array(rsi70_list)
    df['RSI - 30'] = np.array(rsi30_list)
    df['BBD_Delta_Up'] = bbdelta
    df['delta5_8'] = delta5_8
    df['delta8_10'] = delta8_10
    df['delta10_12'] = delta10_12
    df['delta12_15'] = delta12_15
    df['delta15_30'] = delta15_30
    df['delta30_35'] = delta30_35
    df['delta35_40'] = delta35_40
    df['delta40_45'] = delta40_45
    df['delta45_50'] = delta45_50
    df['Stoc_Slowk'] = stoc_slowk
    df['Stoc_Slowd'] = stoc_slowd
    df['KC_High'] = KC_High
    df['KC_Low'] = KC_Low
    df['upper'] = upper
    df['lower'] = lower
    df['var_bollup_kchigh'] = upper-KC_High
    df['var_bolllow_kclow'] = lower-KC_Low
    df['Aroon Up'] = aroonup
    df['Aroon Down'] = aroondown
    df['Delta Aroon'] = aroon
    up = []
    down = []
    df = df.dropna()
    df = boost(df)
    df['%Futur'] = ((df['Close.S']-df['Close']) *100) / (df['Close'])
    df['%Futur2'] = ((df['Close.S2']-df['Close']) *100) / (df['Close'])
    for i in range(0,df.shape[0]-5):
        if df.iloc[i]['%Futur'] > 0.5 :
            up.append(1)
            down.append(0)
        elif df.iloc[i]['%Futur'] < -0.5:
            up.append(0)
            down.append(1)
        else:
            up.append(0)
            down.append(0)
    up.append(0)
    down.append(0)
    up.append(0)
    down.append(0)
    up.append(0)
    down.append(0)
    up.append(0)
    down.append(0)
    up.append(0)
    down.append(0)
    
    
    df['target_up'] = up  
    df['target_down'] = down 
    #df = df.dropna()
    return(df)

def boost(df):
    X = df.copy()
    X = X.drop(['Close'],axis=1)
    X.astype(np.float64)
    X['Close'] = df['Close']
    y = X.iloc[:,-1]
    Xtrain = X.iloc[:-2,:-1]
    Xtest = X.iloc[-2:-1,:-1]
    yshift = y.shift(-1)
    ytrain = yshift.iloc[:-2]
    ytest = yshift.iloc[-2:-1]

    model = xgb.XGBRegressor(n_estimators=20000, learning_rate=1, gamma=1, subsample=1, colsample_bytree=1, max_depth=100,objective='reg:squarederror')

    model.fit( Xtrain, ytrain, early_stopping_rounds=150, eval_set=[(Xtest, ytest)], verbose=0)

    ytrain_pred = model.predict(Xtrain)

    y_pred = model.predict(Xtest)

    pred = model.predict(X.iloc[:,:-1])

    df['Close.S'] = pred
    df['Close.S2'] = df['Close.S']
    df = df.dropna()
    return(df)

def deep_learning(df):
    X = df.iloc[:,:-4]
    y_up = df.iloc[:,-2].values
    y_down = df.iloc[:,-1].values

    scaler = MinMaxScaler(feature_range=(0, 1))
    X = scaler.fit_transform(X)
    y_up = np.array(y_up).reshape(-1,1)
    y_down = np.array(y_down).reshape(-1,1)

    Xtrain = X[:bloc1,:]
    Xtest = X[bloc1:,:]
    ytrain_up = y_up[:bloc1,:]
    ytest_up = y_up[bloc1:,:]
    ytrain_down = y_down[:bloc1,:]
    ytest_down = y_down[bloc1:,:]

    seed = 770
    np.random.seed(seed)

    ytrain_up = ytrain_up.reshape(ytrain_up.shape[0],)
    ytrain_down = ytrain_down.reshape(ytrain_down.shape[0],)

    Xtrain = Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1])

    yhat_up = model_up.predict_classes(Xtest)
    yhat_down = model_down.predict_classes(Xtest)

    predict_up = model_up.predict(Xtest)
    predict_down = model_down.predict(Xtest)

    accuracy_up = accuracy_score(ytest_up, yhat_up)
    accuracy_down = accuracy_score(ytest_down, yhat_down)

    # La précision permet de mesurer la capacité du modèle à refuser résultats non-pertinents : vrais_positifs/(vrais_positifs+faux_positifs)
    precision_up = precision_score(ytest_up, yhat_up)  
    precision_down = precision_score(ytest_down, yhat_down) 


    # Recall : (vrai_positifs/(vrais_positifs+faux_négatifs))
    recall_up = recall_score(ytest_up, yhat_up) 
    recall_down = recall_score(ytest_down, yhat_down) 


    resultats = pd.DataFrame()
    resultats['Date'] = df.index[bloc1:]
    resultats.index= df.index[bloc1:]
    resultats['Move Up'] = yhat_up
    resultats['Confiance up'] = (predict_up)*100
    resultats['Move Down'] = yhat_down
    resultats['Confiance Down'] = (predict_down)*100
    resultats['Actual'] = df.iloc[bloc1:]['Close']
    resultats['Actual.S'] = df.iloc[bloc1:]['Close.S']
    open_S = df['Open'].shift(-1)
    resultats['Open.S'] = open_S.iloc[bloc1:]
    dmp_cp=[]
    dmp_cp = ((resultats['Confiance up']-resultats['Confiance Down'])/(resultats['Confiance up']+resultats['Confiance Down'])*100)
    resultats['DMP_CP'] = dmp_cp
    
    return(resultats,precision_up,precision_down,scaler)

Importing Librairies...
Librairies imported

Global Optimized LumberJack Environment Motor 55
LumberJack Jyss 5780(c)
 °0Oo_D.A.G._26_oO0°
  UNIQUE BT v0.55 



KeyboardInterrupt: 

In [57]:
###################
###### MAIN  ######
###################
DATE = []
TICKER = []
PRICE = []
GREEN = []
RED = []
GREEN_CLOSE = []
RED_CLOSE = []

print('Sraping tickers')
constituents = pd.read_csv(LeChemin+'compteur'+stamp+'.csv')
print('shape avant drop_duplicate: ',constituents.shape)
constituents = constituents.drop_duplicates(['Symb.'])
try:
    constituents = constituents.drop(['Unnamed: 0'],axis=1)
except:
    pass
print('shape après déclonage: ',constituents.shape)    
print('Scrap -----> ok')

#today = str(dt.date.today())[:5]+str(dt.date.today())[5:7]+str(dt.date.today())[7:10]
#startday = str(dt.date.today()-dt.timedelta(1190))[:5]+str(dt.date.today()-dt.timedelta(1190))[5:7]+str(dt.date.today()-dt.timedelta(1190))[7:10]
#yesterday = str(dt.date.today()-dt.timedelta(1))[:5]+str(dt.date.today()-dt.timedelta(1))[5:7]+str(dt.date.today()-dt.timedelta(1))[7:10]
#stamp = input("Enter the date (YYYY-MM-DD): ")

print('Date et heure actuelle : ',time.strftime("%d/%m/%Y %H:%M:%S", time.localtime()))
print('La date scrapée est : ', stamp)



#################
#### LABASE ####
################

LaBase = generbaz()

############################
#### LOAD DEEP LEARNING ####
############################
long = len(constituents)
for loop in range(0, long):

    if loop == 1:
        print('\r',Fore.GREEN,' |==',Fore.BLUE,'O',Fore.RED,'==================|',end='',flush=True)
    if loop == round(long/9):
        print('\r',Fore.GREEN,' |====',Fore.BLUE,'O',Fore.RED,'================|',end='',flush=True)
    if loop == round(long/9)*2:
        print('\r',Fore.GREEN,' |======',Fore.BLUE,'O',Fore.RED,'==============|',end='',flush=True)
    if loop == round(long/9)*3:
        print('\r',Fore.GREEN,' |========',Fore.BLUE,'O',Fore.RED,'============|',end='',flush=True)
    if loop == round(long/9)*4:
        print('\r',Fore.GREEN,' |==========',Fore.BLUE,'O',Fore.RED,'==========|',end='',flush=True)
    if loop == round(long/9)*5:
        print('\r',Fore.GREEN,' |============',Fore.BLUE,'O',Fore.RED,'========|',end='',flush=True)
    if loop == round(long/9)*6:
        print('\r',Fore.GREEN,' |==============',Fore.BLUE,'O',Fore.RED,'======|',end='',flush=True)
    if loop == round(long/9)*7:
        print('\r',Fore.GREEN,' |================',Fore.BLUE,'O',Fore.RED,'====|',end='',flush=True)
    if loop == round(long/9)*8:
        print('\r',Fore.GREEN,' |==================',Fore.BLUE,'O',Fore.RED,'==|',end='',flush=True)

    ticker = (constituents.iloc[loop]['Symb.'])

    yamlup = (LeChemin+'Save_'+ticker+'_up.yaml')
    yamldown = (LeChemin+'Save_'+ticker+'_down.yaml')
    modelup = (LeChemin+'Save_'+ticker+'_up.h5')
    modeldown = (LeChemin+'Save_'+ticker+'_down.h5')

    yaml_file_up = open(yamlup, 'r')
    yaml_file_down = open(yamldown, 'r')
    model_yaml_up = yaml_file_up.read()
    model_yaml_down = yaml_file_down.read()
    yaml_file_up.close()
    yaml_file_down.close()
    model_up = model_from_yaml(model_yaml_up)
    model_down = model_from_yaml(model_yaml_down)
    # load weights into new model
    model_up.load_weights(modelup)
    model_down.load_weights(modeldown)

    df = LaBase[LaBase['Symbol']==ticker]
    df = df.drop(['Symbol'],axis=1)
    
    global delta,bloc1,bloc2
    delta = round(df.shape[0])
    bloc1 = round(delta*0.80)
    bloc2 = delta - bloc1
    
    df = prepa_data(df)

    resultats,precision_up,precision_down,scaler = deep_learning(df)

    price = df.iloc[loop]['Close']


###########################
##### MAKING SIGNALS #####
##########################
    
    for i in range(0,len(resultats)):
    
        if (precision_up * 100) > 69 and (precision_down * 100) > 69:

            filtre_up = 93
            filtre_down = 97
            filtre_up_close = 80
            filtre_down_close = 80


            #plt.figure(figsize=(26,8),dpi=300)
            #plt.title('Deep Learning '+ticker+' Trade Signals')
            #plt.plot(resultats['Actual'],color='purple',lw=0.9,label=ticker)

            if resultats.iloc[i]['Confiance Down'] > filtre_down:
                doob = -1
            elif resultats.iloc[i]['Confiance Down'] > filtre_down_close and resultats.iloc[i]['Confiance Down'] < filtre_down :
                waab = -1
                doob = 0
            elif resultats.iloc[i]['Confiance up'] > filtre_up:
                doob = 1
            elif resultats.iloc[i]['Confiance up'] > filtre_up_close and resultats.iloc[i]['Confiance up'] < filtre_up:
                waab = 1
                doob = 0
            else :
                doob = 0
                waab = 0

            if doob == 1 :
                DATE.append(df.index[i])
                TICKER.append(ticker)
                GREEN.append(1)
                RED.append(0)
                GREEN_CLOSE.append(0)
                RED_CLOSE.append(1)
                PRICE.append(price)

            elif doob == -1 :
                DATE.append(df.index[i])
                TICKER.append(ticker)
                GREEN.append(0)
                RED.append(1)
                GREEN_CLOSE.append(1)
                RED_CLOSE.append(0)
                PRICE.append(price)

            elif doob == 0 and waab == -1 :
                DATE.append(df.index[i])
                TICKER.append(ticker)
                GREEN.append(0)
                RED.append(0)
                GREEN_CLOSE.append(0)
                RED_CLOSE.append(1)
                PRICE.append(price)

            elif doob == 0 and waab == 1 :
                DATE.append(df.index[i])
                TICKER.append(ticker)
                GREEN.append(0)
                RED.append(0)
                GREEN_CLOSE.append(1)
                RED_CLOSE.append(0)
                PRICE.append(price)

            else:
                DATE.append(df.index[i])
                TICKER.append(ticker)
                GREEN.append(0)
                RED.append(0)
                GREEN_CLOSE.append(0)
                RED_CLOSE.append(0)
                PRICE.append(price)


        else:
            DATE.append(df.index[i])
            TICKER.append(ticker)
            GREEN.append(0)
            RED.append(0)
            GREEN_CLOSE.append(0)
            RED_CLOSE.append(0)
            PRICE.append(price)

signals = pd.DataFrame()
signals['Date'] = DATE
signals['Ticker'] = TICKER
signals['Open Long'] = GREEN
signals['Close Long'] = GREEN_CLOSE
signals['Open Short'] = RED
signals['Close Short'] = RED_CLOSE 
signals['Close Price'] = PRICE

try:

    signals = pd.read_csv('RESULTS/'+LeNom+'SIGNALS_'+stamp+'.csv')
    try:
        signals = signals_csv.drop('Unnamed: 0',axis=1)
    except:
        pass
    print('Le fichier SIGNALS_'+stamp+'.csv existe déjà')


except:
    print('\n Création du fichier SIGNALS_'+stamp+'.csv')
    try:
        signals = signals_csv.drop('Unnamed: 0',axis=1)
    except:
        pass
    signals.to_csv('RESULTS/'+LeNom+'SIGNALS_'+stamp+'.csv')


Sraping tickers
shape avant drop_duplicate:  (287, 6)
shape après déclonage:  (287, 5)
Scrap -----> ok
Date et heure actuelle :  30/10/2019 21:44:05
La date scrapée est :  2018-12-31
 ON GENERE LaBase 
   |==================  O  ==| 
 Time for completing LaBase Generation = 358.240000 seconds 
 
Il y a  0  mauvais tickers 

Les voici : 
[]
Base générée :  (235326, 6)
Il y a  (1, 6)  Duplicats
Base déclonée :  (235325, 6)
   |==================  O  ==|Le fichier SIGNALS_2018-12-31.csv existe déjà


In [59]:
########################
####### BACKTEST########
########################
pos_long = 0
pos_short = 0
pactol = 400000
price_buy = 0
price_sell = 0
x = 0
winners = []
loosers = []
nb_win = 0
nb_los = 0
mini_pnl = 0
pos_duration = []
average_duration = resultats.iloc[0,0] - resultats.iloc[0,0]
trades = []
eq_curx = []
eq_cury = []
maxdd = []
nbS = 0
nbL = 0

for i in range(0,len(resultats)-1):
    if pos_long == 0 and pos_short == 0 and resultats.iloc[i]['Open Long'] == 1 :
        pos_long = 1
        nbL+=1
        price_buy = round(resultats.iloc[i]['Open.S'],2)
        x = round((pactol * 0.0025)/price_buy)
        time_enter = resultats.iloc[i]['Date']

        signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 1,\
                           'Close Long' : 0, 'Open Short' : 0, 'Close Short' : 0,\
                           'Buy/Sell Price' : price_buy, 'Quantity' : x, 'PNL' : 0,\
                                   'Cumulative PNL':signals.iloc[:i]['PNL'].sum(),\
                                   'Latent PNL':0}],ignore_index=True)

        print('\n Le ',str(resultats.iloc[i]['Date']),', achat de ',x,' actions à',round(resultats.iloc[i]['Open.S'],2),'$')
        print('Nouveau pactol : ',round(pactol,2))
        trades.append((round(resultats.iloc[i]['Open.S'],2),resultats.iloc[i]['Date'],1))

    elif pos_long == 1 and resultats.iloc[i]['Close Long'] == 0 :

        signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                           'Close Long' : 0, 'Open Short' : 0, 'Close Short' : 0,\
                           'Buy/Sell Price' : price_buy, 'Quantity' : x, 'PNL' : 0,\
                                   'Cumulative PNL':signals.iloc[:i]['PNL'].sum(),\
                                   'Latent PNL':round((resultats.iloc[i]['Open.S'] - price_buy)*x,2)}],ignore_index=True)
        try:
            maxdd.append(round((resultats.iloc[i]['Open.S'] - price_buy)*x,2))
        except:
            print('Null val of maxdd')
            maxdd.append('Null')


    elif pos_long == 1 and resultats.iloc[i]['Close'] == 1 :
        old_pactol = round(pactol,2)
        eq_curx.append(resultats.iloc[i]['Date'])
        eq_cury.append(old_pactol)
        pos_long = 0
        price_sell = round(resultats.iloc[i]['Open.S'],2)
        mini_pnl = round(x*(price_sell-price_buy),2)
        trades.append((round(resultats.iloc[i]['Open.S'],2),resultats.iloc[i]['Date'],2))

        signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                           'Close Long' : 1, 'Open Short' : 0, 'Close Short' : 0,\
                           'Buy/Sell Price' : price_sell, 'Quantity' : x, 'PNL' : mini_pnl,\
                                   'Cumulative PNL':round(signals.iloc[:i]['PNL'].sum(),2),'Latent PNL':0}],ignore_index=True)

        pactol = round((pactol + mini_pnl),2)
        print('\n Le ',str(resultats.iloc[i]['Date']),', vente pour close de ',x,' actions à',round(resultats.iloc[i]['Open.S'],2),'$')
        print('Nouveau pactol : ',pactol)
        print('pnl:', mini_pnl)
        print('prix de vente',price_sell)
        print('prix d achat',price_buy)

        pos_duration.append(resultats.iloc[i]['Date'] - time_enter)

        if mini_pnl > 0:
            winners.append(mini_pnl)
            mini_pnl = 0
            nb_win +=1
        else :
            loosers.append(mini_pnl)
            mini_pnl = 0
            nb_los +=1


    elif pos_short == 0 and pos_long == 0 and resultats.iloc[i]['Open Short']:
        pos_short = 1
        nbS+=1
        price_sell = round(resultats.iloc[i]['Open.S'],2)
        x = round((pactol * 0.0025)/price_sell)
        time_enter = resultats.iloc[i]['Date']

        signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                           'Close Long' : 0, 'Open Short' : 1, 'Close Short' : 0,\
                           'Buy/Sell Price' : price_sell, 'Quantity' : x, 'PNL' : 0,\
                                   'Cumulative PNL':signals.iloc[:i]['PNL'].sum(),'Latent PNL':0}],ignore_index=True)

        print('\n Le ',str(resultats.iloc[i]['Date']),', vente de ',x,' actions à',round(resultats.iloc[i]['Open.S'],2),'$')
        print('Nouveau pactol : ',pactol)
        trades.append((round(resultats.iloc[i]['Open.S'],2),resultats.iloc[i]['Date'],-1))


    elif pos_short == 1 and resultats.iloc[i]['Close Short'] == 0 :

        signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                           'Close Long' : 0, 'Open Short' : 0, 'Close Short' : 0,\
                           'Buy/Sell Price' : price_sell, 'Quantity' : x, 'PNL' : 0,\
                                   'Cumulative PNL':signals.iloc[:i]['PNL'].sum(),\
                                   'Latent PNL':-round((resultats.iloc[i]['Open.S'] - price_sell)*x,2)}],ignore_index=True)
        try:
            maxdd.append(round((resultats.iloc[i]['Open.S'] - price_sell)*x,2))
        except:
            print('Null val of maxdd')
            maxdd.append(0)

    elif pos_short == 1 and resultats.iloc[i]['Close Short'] == 1 :
        old_pactol = round(pactol,2)
        eq_curx.append(resultats.iloc[i]['Date'])
        eq_cury.append(old_pactol)
        pos_short = 0
        price_buy = round(resultats.iloc[i]['Open.S'],2)
        trades.append((round(resultats.iloc[i]['Open.S'],2),resultats.iloc[i]['Date'],-2))
        mini_pnl = round(x*(price_sell-price_buy),2)

        signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                           'Close Long' : 0, 'Open Short' : 0, 'Close Short' : 1,\
                           'Buy/Sell Price' : price_buy, 'Quantity' : x, 'PNL' : mini_pnl,\
                                   'Cumulative PNL':signals.iloc[:i]['PNL'].sum(),'Latent PNL':0}],ignore_index=True)

        pactol = round((pactol + mini_pnl),2)
        print('\n Le ',str(resultats.iloc[i]['Date']),', achat pour close de ',x,' actions à',round(resultats.iloc[i]['Open.S'],2),'$')
        print('Nouveau pactol : ',pactol)

        print('prix de vente',price_sell)
        print('prix d achat',price_buy)

        print('pnl:', mini_pnl)
        pos_duration.append(resultats.iloc[i]['Date'] - time_enter)
        if mini_pnl > 0:
            winners.append(mini_pnl)
            mini_pnl = 0
            nb_win +=1
        else :
            loosers.append(mini_pnl)
            mini_pnl = 0
            nb_los +=1
    else:

        signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                       'Close Long' : 0, 'Open Short' : 0, 'Close Short' : 0,\
                       'Buy/Sell Price' : 0, 'Quantity' : 0, 'PNL' : 0,\
                                   'Cumulative PNL':signals.iloc[:i]['PNL'].sum(),'Latent PNL':0}],ignore_index=True)

if pos_long == 1:
    pos_long = 0
    print('Cutting non resolved position')
    print(' Le price buy était de : ',price_buy)
    price_sell = round(resultats.iloc[i]['Open.S'],2)
    x = round((pactol * 0.0025)/price_sell)
    print('x = ',x)
    print('Et le price sell : ',price_sell)
    mini_pnl = round(x*(price_sell-price_buy),2)

    signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                           'Close Long' : 1, 'Open Short' : 0, 'Close Short' : 0,\
                           'Buy/Sell Price' : price_sell, 'Quantity' : x, 'PNL' : mini_pnl,\
                               'Cumulative PNL':signals.iloc[:i]['PNL'].sum(),'Latent PNL':0}],ignore_index=True)

    pactol = round((pactol + mini_pnl),2)
    print('pnl : ',mini_pnl)
    if mini_pnl > 0:
        winners.append(mini_pnl)
        mini_pnl = 0
        nb_win +=1
    else :
        loosers.append(mini_pnl)
        mini_pnl = 0
        nb_los +=1


if pos_short == 1:
    pos_short = 0
    print('Cutting non resolved position')
    print(' Le price sell était de : ',price_sell)
    price_buy = round(resultats.iloc[i]['Open.S'],2)
    x = round((pactol * 0.0025)/price_buy)
    print('x = ',x)
    print('Et le price_buy : ',price_buy)
    mini_pnl = round(x*(price_sell-price_buy),2)

    signals = signals.append([{ 'Date': resultats.iloc[i]['Date'], 'Ticker' : ticker, 'Open Long' : 0,\
                           'Close Long' : 0, 'Open Short' : 0, 'Close Short' : 1,\
                           'Buy/Sell Price' : price_buy, 'Quantity' : x, 'PNL' : mini_pnl,\
                               'Cumulative PNL':signals.iloc[:i]['PNL'].sum(),'Latent PNL':0}],ignore_index=True)

    pactol = round((pactol + mini_pnl),2)
    print('pnl : ',mini_pnl)

    if mini_pnl > 0:
        winners.append(mini_pnl)
        mini_pnl = 0
        nb_win +=1
    else :
        loosers.append(mini_pnl)
        mini_pnl = 0
        nb_los +=1


pnl = round(pactol - (pactol * 0.0025),2)
print('\n Begining of BackTest :',resultats.iloc[0]['Date'])
print('Instrument :',ticker)
print('Face value per trade : $',round(pactol * 0.0025))
print('End of BackTest :',resultats.iloc[-1]['Date'])
duration = pd.to_timedelta((resultats.iloc[-1]['Date'] - resultats.iloc[0]['Date']),unit='d')
print("BackTest's period :",duration)
print('Pnl :',round(pnl,2),'$')
print('% Pnl : ',round(pnl/(pactol * 0.0025)*100,2))
print('Total winners :',round(sum(winners),2),'$')
print('Total loosers :',round(sum(loosers),2),'$')
print('Number of winners :',round(nb_win))
print('Number of loosers :',round(nb_los))
print('Maximum Drawdown : ',round(min(maxdd),2))
print('% Max Drawdown : ',round(min(maxdd)/(pactol * 0.0025)*100,2),'%')
print('Nombre toatl de trades : ',round(nbL+nbS,2))
print('% de trades Long : ',round((nbL*100)/(nbL+nbS),2))
print('% de trades Short : ',round((nbS*100)/(nbL+nbS),2))
try:
    print('% winners :', round((nb_win/(nb_win+nb_los) * 100),2),'%')
    print('Average winners :',(round(sum(winners)/(nb_win))))
except:
    print('% winners : null')
try:
    print('Average losers :',(round(sum(loosers)/(nb_los))))
except:
    print('Average losers : Inf')
try:
    for t in range(0,len(pos_duration)):
        average_duration = pd.to_timedelta((average_duration + pos_duration[t]),unit='d')
    average_duration = average_duration/(nb_los+nb_win)
    print('Average Duration :', average_duration)
except:
    print('No Average Duration available')
try:
    print('Profit Factor : ',abs(round((sum(winners)/sum(loosers)),2)))
except:
    print('Profit Factor : null')

########################
##### PLOT TRADES ######
########################

plt.figure(figsize=(26,8),dpi=300)
plt.title('Deep Learning '+ticker+' Trades')
plt.plot(resultats['Open.S'],color='blue',lw=0.9)
for i in range(0,len(trades)):
    if trades[i][2] == 1:
        plt.scatter(x=trades[i][1],y=trades[i][0],c='green',marker='^',lw=2.5)
    elif trades[i][2] == -1:
        plt.scatter(x=trades[i][1],y=trades[i][0],c='red',marker='^',lw=2.5)
    elif trades[i][2] == 2:
        plt.scatter(x=trades[i][1],y=trades[i][0],c='g',marker='x',lw=2.5)
    elif trades[i][2] == -2:
        plt.scatter(x=trades[i][1],y=trades[i][0],c='r',marker='x',lw=2.5)

plt.scatter(x=resultats.index[i],y=resultats.iloc[i]['Open.S'],c='g',marker='^',label='Open Long')
plt.scatter(x=resultats.index[i],y=resultats.iloc[i]['Open.S'],c='r',marker='^',label='Open Short')
plt.scatter(x=resultats.index[i],y=resultats.iloc[i]['Open.S'],c='g',marker='x',label='Close Long')
plt.scatter(x=resultats.index[i],y=resultats.iloc[i]['Open.S'],c='r',marker='x',label='Close Short')
plt.legend()
plt.savefig('backtest de '+str(ticker)+'_02.pdf')
plt.close()

#########################
##### PLOT E-CURVE ######
#########################
plt.figure(figsize=(26,8),dpi=300)
plt.title('Deep Learning '+ticker+' Equity Curve')
plt.plot(eq_curx,eq_cury)
plt.scatter(x=eq_curx,y=eq_cury,c='orange',marker='o',lw=2.5)
plt.savefig('backtest de '+str(ticker)+'_03.pdf')
plt.close()


f.close()
sys.stdout = orig_stdout

print('\r Boucle ', loop, 'terminée...',end='')
print('Opération restantes :', len(constituents)-loop,'/',len(constituents),'\n')


backtest = backtest.append([{'Symb.':ticker,'Name':name,'Sector':sector,'Period':duration,'Begin.':resultats.iloc[0,0],'End':resultats.iloc[-1,0],\
                  'Pnl $':pnl,'Pnl %':round(pnl/(pactol * 0.0025)*100,2),'Gross Profit':round(sum(winners),2),'Gross Loss':round(sum(loosers),2),'# Winners':nb_win,'# Loosers':nb_los,\
                  '% Winners':round((nb_win/(nb_win+nb_los) * 100),2),'MaxDD $':round(min(maxdd),2),'MaxDD %':round(min(maxdd)/(pactol * 0.0025)*100,2),'Aver. Win $':(round(sum(winners)/(nb_win))),\
                  'Aver. Loos $':(round(sum(loosers)/(nb_los+0.001))),'Aver. Duration':average_duration,\
                  'Total Trades':nbL+nbS,'% Long':round((nbL*100)/(nbL+nbS),2),'% Short':round((nbS*100)/(nbL+nbS),2), 'Profit Factor':abs(round((sum(winners)/sum(loosers)),2))}], ignore_index=True)


#signals = signals.append([{ 'Date': list_signals[0], 'Ticker' : list_signals[8], 'Open Long' : list_signals[1],\
                           #'Close Long' : list_signals[2], 'Open Short' : list_signals[3], 'Close Short' : list_signals[4],\
                           #'Buy/Sell Price' : list_signals[5], 'Quantity' : list_signals[6], ' tPNL' : list_signals[7]}])

#signals['Date'] = list_signals[0]
#signals['Ticker'] = list_signals[8]
#signals['Open Long'] = list_signals[1]
#signals['Close Long'] = list_signals[2]
#signals['Open Short'] = list_signals[3]
#signals['Close Short'] = list_signals[4]
#signals['Buy/Sell Price'] = list_signals[5]
#signals['Quantity'] = list_signals[6]
#signals['tPNL'] = list_signals[7]

# symbol / name / sector / period of backtest / debut / fin / pnl / total winners / total loosers / Nb winners / nb loosers / 
# / % winners / Max DD / Aver Win / Aver Loos / Average duration / Profit Factor
print(Fore.YELLOW,'Writing BACKTEST CSV AND SIGNALS.CSV',Style.RESET_ALL)
backtest.to_csv('BackTest.csv')
#signals.sort_values(by=['Date'])
signals.to_csv('SIGNALS.csv')

print(Fore.CYAN,'BACKTEST CSV WRITED AND SIGNALS.CSV',Style.RESET_ALL)
#except:
 #   print(Fore.RED,'Problème loop : ',loop,Style.RESET_ALL)
  #  error.append((loop,ticker))
   #  
    #continue
print('Done!')

KeyError: 'Open Long'

In [60]:
resultats.head(0)

Empty DataFrame
Columns: [Date, Move Up, Confiance up, Move Down, Confiance Down, Actual, Actual.S, Open.S, DMP_CP]
Index: []

In [62]:
signals.head(0)

Empty DataFrame
Columns: [Unnamed: 0, Date, Ticker, Open Long, Close Long, Open Short, Close Short, Close Price]
Index: []

Verifier que ça joue bien l open

verifier la difference entre BT et Live au nieveau de la prise de date.

=> Passer par des fonctions pour plus de parties?


### LiveTest

[ 1 3 4 5 6 7 8 9 10]

Je prends de 1 à 10, => signal. Puis le lendemain, le 11 apparait

[1 2 3 4 5 6 7 8 9 10 100]

JE vire le 1, et je refais.


### BackTest

[1 2 3 4 5 6 7 8 9 0 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25]

Il faut donc prendre de 1 à 10 et etablir le signal
Puis de 2 à 11, etc...


In [66]:
LaBase.shape

(235325, 6)

In [67]:
LaBase.head()

Symbol       High        Low       Open     Volume      Close
Date                                                                    
2015-09-29      A  34.060001  33.240002  33.360001  2252400.0  32.474979
2015-09-30      A  34.430000  33.669998  34.080002  2545700.0  33.042850
2015-10-01      A  34.480000  33.480000  34.320000  3011400.0  32.474979
2015-10-02      A  34.680000  33.119999  33.180000  3519400.0  33.370094
2015-10-05      A  35.389999  34.840000  35.040001  1675400.0  34.014977

In [68]:
LaBase.tail()

Symbol        High         Low        Open     Volume       Close
Date                                                                        
2018-12-24    ZBH   99.379997   96.989998   99.040001   931600.0   96.445541
2018-12-26    ZBH  102.080002   97.300003   97.599998  1619200.0  101.196892
2018-12-27    ZBH  101.730003   98.150002  100.250000  1578900.0  101.127289
2018-12-28    ZBH  103.989998  101.559998  102.000000  1860000.0  102.061905
2018-12-31    ZBH  104.389999  102.430000  103.040001  1389700.0  103.125778

In [73]:
len(LaBase.index.unique())

825

In [79]:
for foo in range(0,long):
    if len(LaBase[LaBase['Symbol']==ticker]) != 820:
        print(ticker,' => ', len(LaBase[LaBase['Symbol']==ticker]))